In [1]:
import pandas as pd
import seaborn as sns
import codecs
import os
import math
import re
import string
from collections import Counter, defaultdict

sample_file = "../data/wpdx_sample_data.csv"
file = "../../Documents/waterpoint/Water_Point_Data_Exchange_Complete_Dataset.csv"

In [2]:
os.path.exists(file)

True

In [3]:
data = pd.read_csv(file)
data.head()

/Users/cjiahui/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (2,5,7,17,18,19,27,29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Row ID,#country_name,#water_source,#water_tech,#status_id,#management,#pay,#installer,#install_year,#status,...,#converted,#created,#updated,#lat_deg,#lon_deg,Location,Count,#fecal_coliform_presence,#fecal_coliform_value,#subjective_quality
0,1,"Tanzania, United Republic of",Spring,Gravity Communal standpipe,no,NaN,Never pay,RWE,1972.0,Status:Not functional|Quantity:Dry|Quality:Soft,...,"""#water_tech, #country_id, #data_lnk""",10/23/2015 01:03:07 AM +0000,10/23/2015 01:03:07 AM +0000,-2.83257,36.3090,"(-2.83257, 36.309)",1,NaN,NaN,NaN
1,2,"Tanzania, United Republic of",Rainwater harvesting,Gravity Communal standpipe,yes,NaN,Never pay,World Vision,2005.0,Status:Functional|Breakdown Year:2007|Reason N...,...,"""#water_tech, #country_id, #data_lnk""",10/23/2015 01:03:07 AM +0000,10/23/2015 01:03:07 AM +0000,-2.84781,36.5284,"(-2.84781, 36.5284)",1,NaN,NaN,NaN
2,3,"Tanzania, United Republic of",Rainwater harvesting,Gravity Communal standpipe,yes,NaN,Never pay,World Vision,2005.0,Status:Functional|Quantity:Seasonal|Quality:Soft,...,"""#water_tech, #country_id, #data_lnk""",10/23/2015 01:03:07 AM +0000,10/23/2015 01:03:07 AM +0000,-2.84796,36.5288,"(-2.84796, 36.5288)",1,NaN,NaN,NaN
3,4,"Tanzania, United Republic of",Rainwater harvesting,Gravity Communal standpipe,yes,NaN,Never pay,World Vision,2005.0,Status:Functional|Quantity:Seasonal|Quality:Soft,...,"""#water_tech, #country_id, #data_lnk""",10/23/2015 01:03:07 AM +0000,10/23/2015 01:03:07 AM +0000,-2.85318,36.5276,"(-2.85318, 36.5276)",1,NaN,NaN,NaN
4,5,Nicaragua,NaN,NaN,yes,NaN,NaN,NaN,2011.0,NaN,...,"#data_lnk, #lon_deg, #lat_deg, #country_id",04/08/2016 08:35:49 AM +0000,04/08/2016 08:35:49 AM +0000,12.74070,-86.0121,"(12.7407, -86.0121)",1,NaN,NaN,NaN


In [5]:
mapping_file = "../data/Water.Tech.And.Source.xlsx"
source = pd.read_excel(mapping_file, sheet_name="Water Sources Matched")
tech = pd.read_excel(mapping_file, sheet_name='Water Techs Matched')

In [6]:
tech

,#water_tech,Count,Water Tech,Water Source,Unnamed: 4,Unnamed: 5,Unnamed: 6,Matched,Total,Unnamed: 9,Percent Matched
0,Deep borehole,25959,NaN,Borehole,NaN,NaN,NaN,25959,25959,NaN,0.888498
1,Afridev,25247,Hand Pump - Afridev,NaN,NaN,NaN,NaN,25247,25247,NaN,NaN
2,Protected spring,24280,NaN,Protected Spring,NaN,NaN,NaN,24280,24280,NaN,NaN
3,DUG WELL,20568,NaN,Shallow Well,NaN,NaN,NaN,20568,20568,NaN,NaN
4,Bush Pump Type B,20338,Hand Pump,NaN,NaN,NaN,NaN,20338,20338,NaN,NaN
5,Standpipe,20247,Tapstand,NaN,NaN,NaN,NaN,20247,20247,NaN,NaN
6,Shallow well,16717,NaN,Shallow Well,NaN,NaN,NaN,16717,16717,NaN,NaN
7,India Mark,16402,Hand Pump - India Mark II,NaN,NaN,NaN,NaN,16402,16402,NaN,NaN
8,Bucket,15669,Bucket,NaN,NaN,NaN,NaN,15669,15669,NaN,NaN
9,Rainwater Harvest Tank,15417,NaN,Rainwater Harvesting,NaN,NaN,NaN,15417,15417,NaN,NaN


In [9]:
dirty_tech_2techsource = dict()
SEPARATOR = "~"

for i, row in tech.iterrows():
    water_source = "NA" if pd.isnull(row['Water Source']) else row['Water Source']
    water_tech = "NA" if pd.isnull(row['Water Tech']) else row['Water Tech']
    dirty_tech_2techsource[row['#water_tech']] = water_tech + SEPARATOR + water_source
    

In [10]:
dirty_tech_2techsource

{'Deep borehole': 'NA~Borehole',
 'Afridev': 'Hand Pump - Afridev ~NA',
 'Protected spring': 'NA~Protected Spring ',
 'DUG WELL': 'NA~Shallow Well',
 'Bush Pump Type B': 'Hand Pump~NA',
 'Standpipe': 'Tapstand ~NA',
 'Shallow well': 'NA~Shallow Well',
 'India Mark': 'Hand Pump - India Mark II ~NA',
 'Bucket': 'Bucket~NA',
 'Rainwater Harvest Tank': 'NA~Rainwater Harvesting',
 'Borne-fontaine': 'Tapstand ~NA',
 'INDUS': 'Hand Pump~NA',
 'AfriDev': 'Hand Pump - Afridev ~NA',
 'Bucket Only': 'Bucket~NA',
 'Public Stand Post': 'Tapstand ~NA',
 'TUBE WELL': 'NA~Borehole',
 'Yard tap for public use': 'Tapstand ~NA',
 'Vergnet HPV 60': 'Hand Pump - Vergnet ~NA',
 'Gravity Communal standpipe': 'Tapstand ~NA',
 'PAMIR': 'Hand Pump~NA',
 'Ghana modified India Mark II': 'Hand Pump - India Mark II ~NA',
 'Kardia': 'Hand Pump~NA',
 'Gravity': 'NA~NA',
 'PHE 6 Handpump': 'Hand Pump~NA',
 'Non-Mechanized Borehole': 'NA~Borehole',
 'Other': 'NA~NA',
 'Hand Pump - Nira/TaHand Pump - Nira Hand pump': 'H

In [11]:
dirty_source_2techsource = dict()
SEPARATOR = "~"

for i, row in source.iterrows():
    water_source = "NA" if pd.isnull(row['Water Source']) else row['Water Source']
    water_tech = "NA" if pd.isnull(row['Water Tech']) else row['Water Tech']
    dirty_source_2techsource[row['#water_source']] = water_tech + SEPARATOR + water_source

In [12]:
dirty_source_2techsource

{' Bottled Water': 'NA~NA',
 ' Dam': 'NA~Surface Water (River/Stream/Lake/Pond/Dam)',
 ' Lake, pond, or other standing water source': 'NA~Surface Water (River/Stream/Lake/Pond/Dam)',
 ' River, stream, or other flowing water source': 'NA~Surface Water (River/Stream/Lake/Pond/Dam)',
 ' Shifting well': 'NA~NA',
 0: 'NA~NA',
 '1Dam/pan (runoff harvesting)': 'NA~Surface Water (River/Stream/Lake/Pond/Dam)',
 '1Surface water (lake/river/stream)': 'NA~Surface Water (River/Stream/Lake/Pond/Dam)',
 'AUGER': 'NA~NA',
 'Above Ground Tank': 'NA~NA',
 'Any': 'NA~NA',
 'Artesian drilling': 'NA~NA',
 'Artesian well': 'NA~NA',
 'Artesian wells': 'NA~NA',
 'Artsian Wells': 'NA~NA',
 'Automatic pump': 'NA~NA',
 'Borehole': 'NA~Borehole',
 'Borehole ': 'NA~Borehole',
 'Borehole  with sola system': 'NA~Borehole - Mechanized with solar',
 'Borehole Pump To Tap Water': 'Tapstand~Borehole',
 'Borehole fitted solar system': 'NA~Borehole - Mechanized with solar',
 'Borehole fitted with Manual pump': 'Hand Pump~

## Adding columns from dirty tech mapping

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478987 entries, 0 to 478986
Data columns (total 30 columns):
Row ID                      478987 non-null int64
#country_name               478699 non-null object
#water_source               304097 non-null object
#water_tech                 402497 non-null object
#status_id                  478987 non-null object
#management                 196375 non-null object
#pay                        185664 non-null object
#installer                  177108 non-null object
#install_year               354612 non-null float64
#status                     307105 non-null object
#source                     478987 non-null object
#adm1                       392575 non-null object
#adm2                       376620 non-null object
#wpdx_id                    478987 non-null object
#report_date                478987 non-null object
#country_id                 478684 non-null object
#activity_id                377157 non-null object
#data_lnk             

In [14]:
# dirty_tech_2techsource
dirty_tech_values = []

for i, row in data.iterrows():
    water_tech = row['#water_tech']
    if water_tech in dirty_tech_2techsource:
        dirty_tech_values.append(dirty_tech_2techsource[water_tech])
    else:
        dirty_tech_values.append("NO_MAPPING")

In [23]:
# dirty_source_2techsource
dirty_source_values = []

for i, row in data.iterrows():
    water_source = row['#water_source']
    if water_source in dirty_source_2techsource:
        dirty_source_values.append(dirty_source_2techsource[water_source])
    else:
        dirty_source_values.append("NO_MAPPING")

In [21]:
# 1 is from dirty tech mappings
# 2 is from dirty source mappings

data['clean_water_tech1'] = [val.split(SEPARATOR)[0] if SEPARATOR in val else val for val in dirty_tech_values]
data['clean_water_source1'] = [val.split(SEPARATOR)[1] if SEPARATOR in val else val for val in dirty_tech_values]

In [24]:
data['clean_water_tech2'] = [val.split(SEPARATOR)[0] if SEPARATOR in val else val for val in dirty_source_values]
data['clean_water_source2'] = [val.split(SEPARATOR)[1] if SEPARATOR in val else val for val in dirty_source_values]

### Reconcile dirty tech and dirty source mappings

- if conflict in clean water tech1 and 2, choose 1
- if conflict in clean water source1 and 2, choose 2
- if both are NA, give NA

In [32]:
clean_water_tech = []
clean_water_source = []

for i, row in data.iterrows():
    
    # resolve tech. can refactor to remove first 'if'
    
    if row.clean_water_tech1 == row.clean_water_tech2:
        water = row.clean_water_tech1
    elif row.clean_water_tech1 != "NA" and row.clean_water_tech2 != "NA":
        water = row.clean_water_tech1
    elif row.clean_water_tech2 == "NA":
        water = row.clean_water_tech1
    elif row.clean_water_tech1 == "NA":
        water = row.clean_water_tech2
        
    # resolve source. can refactor to remove first 'if'
    
    if row.clean_water_source1 == row.clean_water_source2:
        source = row.clean_water_source2
    elif row.clean_water_source1 != "NA" and row.clean_water_source2 != "NA":
        source = row.clean_water_source2
    elif row.clean_water_source2 == "NA":
        source = row.clean_water_source1
    elif row.clean_water_source1 == "NA":
        source = row.clean_water_source2
        
    
    clean_water_tech.append(water)
    clean_water_source.append(source)

In [33]:
data['clean_water_tech'] = clean_water_tech
data['clean_water_source'] = clean_water_source

In [34]:
data

,Row ID,#country_name,#water_source,#water_tech,#status_id,#management,#pay,#installer,#install_year,#status,...,Count,#fecal_coliform_presence,#fecal_coliform_value,#subjective_quality,clean_water_tech1,clean_water_source1,clean_water_tech2,clean_water_source2,clean_water_tech,clean_water_source
0,1,"Tanzania, United Republic of",Spring,Gravity Communal standpipe,no,NaN,Never pay,RWE,1972.0,Status:Not functional|Quantity:Dry|Quality:Soft,...,1,NaN,NaN,NaN,Tapstand,NA,NA,Spring,Tapstand,Spring
1,2,"Tanzania, United Republic of",Rainwater harvesting,Gravity Communal standpipe,yes,NaN,Never pay,World Vision,2005.0,Status:Functional|Breakdown Year:2007|Reason N...,...,1,NaN,NaN,NaN,Tapstand,NA,NA,Rainwater Harvesting,Tapstand,Rainwater Harvesting
2,3,"Tanzania, United Republic of",Rainwater harvesting,Gravity Communal standpipe,yes,NaN,Never pay,World Vision,2005.0,Status:Functional|Quantity:Seasonal|Quality:Soft,...,1,NaN,NaN,NaN,Tapstand,NA,NA,Rainwater Harvesting,Tapstand,Rainwater Harvesting
3,4,"Tanzania, United Republic of",Rainwater harvesting,Gravity Communal standpipe,yes,NaN,Never pay,World Vision,2005.0,Status:Functional|Quantity:Seasonal|Quality:Soft,...,1,NaN,NaN,NaN,Tapstand,NA,NA,Rainwater Harvesting,Tapstand,Rainwater Harvesting
4,5,Nicaragua,NaN,NaN,yes,NaN,NaN,NaN,2011.0,NaN,...,1,NaN,NaN,NaN,NA,NA,NA,NA,NA,NA
5,6,Nicaragua,NaN,Spring Capture,yes,NaN,NaN,NaN,2014.0,NaN,...,1,NaN,NaN,NaN,NA,NA,NA,NA,NA,NA
6,7,Nicaragua,NaN,NaN,yes,NaN,NaN,NaN,2012.0,NaN,...,1,NaN,NaN,NaN,NA,NA,NA,NA,NA,NA
7,8,Nicaragua,NaN,Electric Pumping Gravity Flow Water System,yes,NaN,NaN,NaN,2011.0,NaN,...,1,NaN,NaN,NaN,Borehole - Mechanized,NA,NA,NA,Borehole - Mechanized,NA
8,9,Nicaragua,NaN,NaN,yes,NaN,NaN,NaN,2011.0,NaN,...,1,NaN,NaN,NaN,NA,NA,NA,NA,NA,NA
9,10,Nicaragua,NaN,NaN,yes,NaN,NaN,NaN,2015.0,NaN,...,1,NaN,NaN,NaN,NA,NA,NA,NA,NA,NA
